In [1]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
def read_sqlite(dbfile):
    import sqlite3
    from pandas import read_sql_query, read_sql_table

    with sqlite3.connect(dbfile) as dbcon:
        tables = list(read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", dbcon)['name'])
        out = {tbl : read_sql_query(f"SELECT * from {tbl}", dbcon) for tbl in tables}

    return out

In [4]:
db = read_sqlite("../input_files/steel_moment_axial_combos_latest.db")

In [5]:
db.keys()

dict_keys(['Wshapes_Table6_1', 'wShapes'])

In [6]:
w_shapes = db['wShapes']
w_shapes.shape

(2589600, 9)

In [7]:
Wshapes_Table6_1 = db['Wshapes_Table6_1']
Wshapes_Table6_1.shape

(6139000, 9)

In [8]:
w_shapes.head()

,profile,length_ft,weight_plf,Cb,Pr_over_Pc,Pr_k,Pc_k,Mmax_ftk,Mc_ftk
0,W44X290,0,290.63328,1.0,0.0,0.0,2397.118549,3517.964072,3517.964072
1,W44X290,0,290.63328,1.1,0.0,0.0,2397.118549,3517.964072,3517.964072
2,W44X290,0,290.63328,1.2,0.0,0.0,2397.118549,3517.964072,3517.964072
3,W44X290,0,290.63328,1.3,0.0,0.0,2397.118549,3517.964072,3517.964072
4,W44X290,0,290.63328,1.4,0.0,0.0,2397.118549,3517.964072,3517.964072


In [9]:
Wshapes_Table6_1.head()

,profile,length_ft,weight_plf,Cb,Pr_over_Pc,Pr_k,Pc_k,Mmax_ftk,Mc_ftk
0,W44X335,0,335,1.0,0.0,0.0,2890.17,4040.4,4040.4
1,W44X335,0,335,1.1,0.0,0.0,2890.17,4040.4,4040.4
2,W44X335,0,335,1.2,0.0,0.0,2890.17,4040.4,4040.4
3,W44X335,0,335,1.3,0.0,0.0,2890.17,4040.4,4040.4
4,W44X335,0,335,1.4,0.0,0.0,2890.17,4040.4,4040.4


In [10]:
primary_keys = ['profile', 'length_ft', 'Cb', 'Pr_over_Pc']

merged_df = w_shapes.merge(Wshapes_Table6_1, on=primary_keys, suffixes=('_w_shapes', '_Wshapes_Table6_1'))

In [11]:
merged_df.shape

(514680, 14)

In [12]:
merged_df.head()

,profile,length_ft,weight_plf_w_shapes,Cb,Pr_over_Pc,Pr_k_w_shapes,Pc_k_w_shapes,Mmax_ftk_w_shapes,Mc_ftk_w_shapes,weight_plf_Wshapes_Table6_1,Pr_k_Wshapes_Table6_1,Pc_k_Wshapes_Table6_1,Mmax_ftk_Wshapes_Table6_1,Mc_ftk_Wshapes_Table6_1
0,W44X290,0,290.63328,1.0,0.0,0.0,2397.118549,3517.964072,3517.964072,290,0.0,2398.08,3513.39,3513.39
1,W44X290,0,290.63328,1.1,0.0,0.0,2397.118549,3517.964072,3517.964072,290,0.0,2398.08,3513.39,3513.39
2,W44X290,0,290.63328,1.2,0.0,0.0,2397.118549,3517.964072,3517.964072,290,0.0,2398.08,3513.39,3513.39
3,W44X290,0,290.63328,1.3,0.0,0.0,2397.118549,3517.964072,3517.964072,290,0.0,2398.08,3513.39,3513.39
4,W44X290,0,290.63328,1.4,0.0,0.0,2397.118549,3517.964072,3517.964072,290,0.0,2398.08,3513.39,3513.39


In [13]:
threshold = 2

In [14]:
def calculate_percent_difference(x1, x2):
    if x1 + x2 == 0:
        return 0
    return (abs(x1 - x2) / ((x1 + x2) / 2)) * 100

In [15]:
to_compare_columns = ['Pr_k', 'Pc_k', 'Mmax_ftk', 'Mc_ftk']

passes2 = 0
failures2 = 0
failed_items2 = []

for index, row in merged_df.iterrows():

    differences = [calculate_percent_difference(row[f'{col}_w_shapes'], row[f'{col}_Wshapes_Table6_1']) for col in to_compare_columns]

    if all(diff <= threshold for diff in differences):
        passes2 += 1
    else:
        failures2 += 1
        failed_items2.append(index)

print('Threshold: ', threshold)
print('Number of passes 2:', passes2)
print('Number of failures 2:', failures2)
print("Failed Items 2:")
merged_df.loc[failed_items2]

Threshold:  2
Number of passes 2: 512622
Number of failures 2: 2058
Failed Items 2:


,profile,length_ft,weight_plf_w_shapes,Cb,Pr_over_Pc,Pr_k_w_shapes,Pc_k_w_shapes,Mmax_ftk_w_shapes,Mc_ftk_w_shapes,weight_plf_Wshapes_Table6_1,Pr_k_Wshapes_Table6_1,Pc_k_Wshapes_Table6_1,Mmax_ftk_Wshapes_Table6_1,Mc_ftk_Wshapes_Table6_1
404400,W14X90,0,90.1848,1.0,0.0,0.00000,793.413112,391.716567,391.716567,90,0.00,793.65,381.50,381.5
404401,W14X90,0,90.1848,1.1,0.0,0.00000,793.413112,391.716567,391.716567,90,0.00,793.65,381.50,381.5
404402,W14X90,0,90.1848,1.2,0.0,0.00000,793.413112,391.716567,391.716567,90,0.00,793.65,381.50,381.5
404403,W14X90,0,90.1848,1.3,0.0,0.00000,793.413112,391.716567,391.716567,90,0.00,793.65,381.50,381.5
404404,W14X90,0,90.1848,1.4,0.0,0.00000,793.413112,391.716567,391.716567,90,0.00,793.65,381.50,381.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406915,W14X90,50,90.1848,2.5,0.6,90.68557,151.142616,176.272455,391.716567,90,90.91,151.52,171.67,381.5
406916,W14X90,50,90.1848,2.6,0.6,90.68557,151.142616,176.272455,391.716567,90,90.91,151.52,171.67,381.5
406917,W14X90,50,90.1848,2.7,0.6,90.68557,151.142616,176.272455,391.716567,90,90.91,151.52,171.67,381.5
406918,W14X90,50,90.1848,2.8,0.6,90.68557,151.142616,176.272455,391.716567,90,90.91,151.52,171.67,381.5


In [16]:
threshold = 1

In [17]:
numeric_columns = ['Pr_k', 'Pc_k', 'Mmax_ftk', 'Mc_ftk']
pass_counts = {col: 0 for col in numeric_columns}
fail_counts = {col: 0 for col in numeric_columns}

# Create a list to store the details of failed items
failed_items_details_df = []

# Iterate through each row
for index, row in merged_df.iterrows():
    # Check the difference for each numeric column
    differences = [calculate_percent_difference(row[f'{col}_w_shapes'], row[f'{col}_Wshapes_Table6_1']) for col in numeric_columns]

    # Check if differences for each column are within 5%
    for col, diff in zip(numeric_columns, differences):
        if diff <= threshold:
            pass_counts[col] += 1
        else:
            fail_counts[col] += 1

# Print the number of passes and failures for each column
print('Pass/Fail counts for each column for threshold: ',threshold)
for col in numeric_columns:
    print(f'{col}: Passes={pass_counts[col]}, Failures={fail_counts[col]}')

Pass/Fail counts for each column for threshold:  1
Pr_k: Passes=514000, Failures=680
Pc_k: Passes=513840, Failures=840
Mmax_ftk: Passes=510672, Failures=4008
Mc_ftk: Passes=510672, Failures=4008


## Run below code to print the failing test cases.

Set range as desired for faster processing

    set in : for index, row in merged_df[400000:500000].iterrows():

In [18]:
to_compare_columns = ['Pr_k', 'Pc_k', 'Mmax_ftk', 'Mc_ftk']
passes3 = 0
failures3 = 0
failed_items_details3 = []

for index, row in merged_df[400000:500000].iterrows():
    differences = [calculate_percent_difference(row[f'{col}_w_shapes'], row[f'{col}_Wshapes_Table6_1']) for col in to_compare_columns]

    # Check if all differences are within 5%
    if all(diff <= threshold for diff in differences):
        passes3 += 1
    else:
        failures3 += 1
        # uncomment below to see failing cases

        # print(index)
        # print(to_compare_columns)
        # print(differences)
        # print(row)

        failed_items_details3.append({
            'index': index,
            'differences': {col: diff for col, diff in zip(to_compare_columns, differences)}
        })

# Print the number of passes and failures
print('Threshold: ',threshold)
print('Number of passes 3 for the first 500 rows of merged_df:', passes3)
print('Number of failures 3 for the first 500 rows of merged_df:', failures3)

# Print details of failed items
print("\nFailed Items Details for the specific range of rows in merged_df:")
for failed_item in failed_items_details3:
    print(f"Failed item at index : {failed_item['index']}:")
    for col, diff in failed_item['differences'].items():
        if diff > threshold:
            print(f"  {col}: {diff:.2f}%")


Threshold:  1
Number of passes 3 for the first 500 rows of merged_df: 95632
Number of failures 3 for the first 500 rows of merged_df: 4368

Failed Items Details for the first 500 rows of merged_df:
Failed item at index : 404400:
  Mmax_ftk: 2.64%
  Mc_ftk: 2.64%
Failed item at index : 404401:
  Mmax_ftk: 2.64%
  Mc_ftk: 2.64%
Failed item at index : 404402:
  Mmax_ftk: 2.64%
  Mc_ftk: 2.64%
Failed item at index : 404403:
  Mmax_ftk: 2.64%
  Mc_ftk: 2.64%
Failed item at index : 404404:
  Mmax_ftk: 2.64%
  Mc_ftk: 2.64%
Failed item at index : 404405:
  Mmax_ftk: 2.64%
  Mc_ftk: 2.64%
Failed item at index : 404406:
  Mmax_ftk: 2.64%
  Mc_ftk: 2.64%
Failed item at index : 404407:
  Mmax_ftk: 2.64%
  Mc_ftk: 2.64%
Failed item at index : 404408:
  Mmax_ftk: 2.64%
  Mc_ftk: 2.64%
Failed item at index : 404409:
  Mmax_ftk: 2.64%
  Mc_ftk: 2.64%
Failed item at index : 404410:
  Mmax_ftk: 2.64%
  Mc_ftk: 2.64%
Failed item at index : 404411:
  Mmax_ftk: 2.64%
  Mc_ftk: 2.64%
Failed item at index :